In [3]:
from yolov7.api_detect import API_Detect

ModuleNotFoundError: No module named 'torch'

In [4]:
from api_utils.utils_file import parse_cfg
cfg = parse_cfg('cfg.ini')
DZ_model = API_Detect(device='2', weights=cfg['DZ_model_detc'])

ModuleNotFoundError: No module named 'cv2'

In [5]:
img_path = 'demo_imgs/3_CH251930825_20221102090002.jpg'

In [6]:
import cv2
import math
import numpy as np 
from matplotlib import pyplot as plt
%matplotlib inline
_, number, bbox_list = DZ_model.inference(img_path)  # bbox (x1,x2,y1,y2)
img = cv2.imread(img_path)
img_ = img.copy()

for bbox in bbox_list:
    img_ = cv2.rectangle(img_, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,0,0), 1)
plt.imshow(img_)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# 计算box的中心点和labels
def bbox2sam_points(bbox_list,img_shape):
    points,labels = [],[]
    H,W,_ = img.shape
    cx,cy = W//2,H//2
    max_distancs = float('inf')
    for bbox in bbox_list:
        x1,y1,x2,y2 = bbox
        x_c,y_c = int((x1+x2)/2),int((y1+y2)/2)
        #  保证在圆环内
        distance = math.sqrt((x_c - cx)**2 + (y_c - cy)**2)
        if distance>H/2:
            continue
        
        # 选最近的鸡
        if distance<max_distancs:
            max_distancs = distance
    
            points = [(x_c,y_c),(x1,x2),(y1,y2)]
            labels = [1,0,0]
            best_c = [x_c,y_c]
            #points = [(x_c,y_c)]
            #labels = [1]

    return points,labels,best_c

In [ ]:
points,labels,bbox_c = bbox2sam_points(bbox_list,img.shape)

In [ ]:
#from efficientvit.sam_model_zoo import create_sam_model
import sys
sys.path.append('D:\\code\\sam_chicken_weight\\efficientvit-master')
sys.path.append('D:\\code\\sam_chicken_weight\\efficientvit-master\\segment-anything')
from efficientvit.sam_model_zoo import create_sam_model
from efficientvit.models.efficientvit.sam import EfficientViTSamPredictor
from efficientvit.models.efficientvit.sam import EfficientViTSamAutomaticMaskGenerator
efficientvit_sam = create_sam_model(name="xl1", weight_url="efficientvit-master/assets/checkpoints/sam/xl1.pt")
efficientvit_sam = efficientvit_sam.cuda().eval()
efficientvit_sam_predictor = EfficientViTSamPredictor(efficientvit_sam)
efficientvit_mask_generator = EfficientViTSamAutomaticMaskGenerator(efficientvit_sam)


In [ ]:
def predict_mask_from_point(
        predictor: EfficientViTSamPredictor, point_coords: np.ndarray, point_labels: np.ndarray
) -> np.ndarray:
    masks, iou_predictions, _ = predictor.predict(
        point_coords=point_coords,
        point_labels=point_labels,
        box=None,
        multimask_output=True,
    )

    mask = masks[iou_predictions.argmax()]
    return mask

def run_point(efficientvit_sam, image_np, points, labels):
    predictor = EfficientViTSamPredictor(efficientvit_sam)
    predictor.set_image(image_np)

    point_labels = np.array(labels)
    point_coords = np.stack(np.array(points), axis=0)

    pre_mask = predict_mask_from_point(predictor, point_coords, point_labels)
    return pre_mask


In [ ]:
import random

chicken_mask = run_point(efficientvit_sam, img, points, labels)
chicken_mask = np.array(chicken_mask).astype(np.uint8)

chicken_heatmapshow = np.repeat(np.expand_dims(chicken_mask*255, axis=-1), 3, axis=-1)
add_img = cv2.addWeighted(img, 0.3, chicken_heatmapshow, 0.7, 0)

plt.imshow(add_img)

In [ ]:
from api_utils.utils_file import  insert_time_device,insert_weight_classical_result
img = insert_time_device(img, 'CH251930808', str('20221021090003'))
def mask2polygon(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # 
    print(len(contours)) 
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        #print(largest_contour)
    return largest_contour

print(chicken_mask.shape,img.shape)
chicken_contour =  mask2polygon(chicken_mask)
cv2.drawContours(img, chicken_contour, -1, (0, 255, 0), 3)
print(img.shape)
cheng_weight =155
species = '1456'
img,_ = insert_weight_classical_result(img, bbox_c, cheng_weight, "黄鸡")
img = cv2.resize(img, (960, 540))
plt.imshow(img)

b